In [0]:
%sql
USE CATALOG etl_project;
USE SCHEMA bigquery_db_cohort_db;

/*
Create Gold cohort-level retention table (1-, 2-, 3-month)
Calculates share of customers placing a second order within 1, 2, and 3 months after their first purchase.
Rounded to 4 decimals to preserve analytical precision.
*/

CREATE OR REPLACE TABLE gold_cohort_retention AS
WITH base AS (
    -- Extract cohort month and month gap between first and second order
    SELECT
        cohort_month,
        months_between
    FROM silver_customer_facts
)
SELECT
    cohort_month,
    
    -- % of customers with second order within 1 month
    ROUND(1.0 * COUNT_IF(months_between = 0) / COUNT(*), 4) AS retention_1m,
    
    -- % within 2 months (includes previous bucket)
    ROUND(1.0 * COUNT_IF(months_between IN (0, 1)) / COUNT(*), 4) AS retention_2m,
    
    -- % within 3 months (includes previous buckets)
    ROUND(1.0 * COUNT_IF(months_between IN (0, 1, 2)) / COUNT(*), 4) AS retention_3m

FROM base
GROUP BY cohort_month
ORDER BY cohort_month;